In [1]:
import os

In [2]:
%pwd        

'd:\\projects\\Data-Extraction-and-NLP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\projects\\Data-Extraction-and-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    unzip_data_dir: Path

In [6]:
from src.DataExtractionAndNLP.constants import *
from src.DataExtractionAndNLP.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        # schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir, 
            unzip_data_dir=config.unzip_data_dir
        )

        return data_ingestion_config

In [8]:
from src.DataExtractionAndNLP.utils.common import get_name
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize


class Cleaning:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def clean(self, data):
        filename = get_name(data)

        output_file = os.path.join(self.config.root_dir, filename)

        with open(output_file, 'r', encoding='utf-8') as file:
            text = file.read()

        # Remove all punctuation marks
        text = re.sub(r'[^\w\s]', '', text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(text)
        cleaned_words_list = [word for word in words if word.isalpha() and word.lower() not in stop_words]

        # Join the filtered tokens back into a cleaned string
        cleaned_text = ' '.join(cleaned_words_list)

        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(cleaned_text)
    

In [9]:
data=['https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/', 'div', 'td-post-content tagdiv-type']

data[2]=data[2].replace(" ", ".")

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    clean = Cleaning(config=data_ingestion_config)
    clean.clean(data)
except Exception as e:
    raise e

[2024-08-01 13:59:56,865: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-01 13:59:56,867: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-01 13:59:56,868: INFO: common: created directory at: artifacts]
[2024-08-01 13:59:56,869: INFO: common: created directory at: artifacts/data_ingestion]
